In [12]:
import psycopg2
import pandas as pd
from dotenv import dotenv_values
import duckdb

In [2]:
config = dotenv_values()

In [5]:
conn = psycopg2.connect(
    database=config['POSTGRES_DB'],
    user=config['POSTGRES_USER'],
    password=config['POSTGRES_PASSWORD'],
    port=config['HOST_PORT']
)

In [11]:
df = pd.read_sql("select * from actor_films", conn)

C:\Users\jlpay\AppData\Local\Temp\ipykernel_14920\3284293064.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("select * from actor_films", conn)


In [18]:
with duckdb.connect("./src/db.duckdb") as con:
    con.sql("""CREATE TABLE actor_films (
            actor text,
            actorid text,
            film text,
            year int,
            votes int,
            rating real,
            filmid text
            )""")
    con.sql("""
INSERT INTO actor_films
SELECT * FROM df
""")

In [20]:
with duckdb.connect("./src/db.duckdb") as con:
    con.table("actor_films").show()

┌───────────────┬───────────┬───────────────────────────────┬───────┬───────┬────────┬───────────┐
│     actor     │  actorid  │             film              │ year  │ votes │ rating │  filmid   │
│    varchar    │  varchar  │            varchar            │ int32 │ int32 │ float  │  varchar  │
├───────────────┼───────────┼───────────────────────────────┼───────┼───────┼────────┼───────────┤
│ Fred Astaire  │ nm0000001 │ Ghost Story                   │  1981 │  7731 │    6.3 │ tt0082449 │
│ Fred Astaire  │ nm0000001 │ The Purple Taxi               │  1977 │   533 │    6.6 │ tt0076851 │
│ Fred Astaire  │ nm0000001 │ The Amazing Dobermans         │  1976 │   369 │    5.3 │ tt0074130 │
│ Fred Astaire  │ nm0000001 │ The Towering Inferno          │  1974 │ 39888 │    7.0 │ tt0072308 │
│ Lauren Bacall │ nm0000002 │ Ernest & Celestine            │  2012 │ 18793 │    7.9 │ tt1816518 │
│ Lauren Bacall │ nm0000002 │ The Forger                    │  2012 │  4472 │    5.4 │ tt1368858 │
│ Lauren B

In [34]:
with duckdb.connect("./src/db.duckdb") as con:
    con.sql("""
SELECT
    actorid,
            year,
            filmid,
            count(*)
FROM actor_films
            group by all
having
            count(*) > 1
""").show()

┌─────────┬───────┬─────────┬──────────────┐
│ actorid │ year  │ filmid  │ count_star() │
│ varchar │ int32 │ varchar │    int64     │
├─────────┴───────┴─────────┴──────────────┤
│                  0 rows                  │
└──────────────────────────────────────────┘



In [33]:
with duckdb.connect("./src/db.duckdb") as con:
    con.sql("""
SELECT
    *
FROM actor_films
where
            actorid = 'nm0763928'
            and year = 2008
""").show()

┌─────────────────┬───────────┬────────────────┬───────┬───────┬────────┬───────────┐
│      actor      │  actorid  │      film      │ year  │ votes │ rating │  filmid   │
│     varchar     │  varchar  │    varchar     │ int32 │ int32 │ float  │  varchar  │
├─────────────────┼───────────┼────────────────┼───────┼───────┼────────┼───────────┤
│ Rodrigo Santoro │ nm0763928 │ Che: Part One  │  2008 │ 44468 │    7.2 │ tt0892255 │
│ Rodrigo Santoro │ nm0763928 │ Che: Part Two  │  2008 │ 32407 │    6.9 │ tt0374569 │
│ Rodrigo Santoro │ nm0763928 │ Redbelt        │  2008 │ 20361 │    6.7 │ tt1012804 │
│ Rodrigo Santoro │ nm0763928 │ Lion's Den     │  2008 │  2987 │    7.0 │ tt1022606 │
│ Rodrigo Santoro │ nm0763928 │ Os Desafinados │  2008 │   397 │    6.1 │ tt0456122 │
└─────────────────┴───────────┴────────────────┴───────┴───────┴────────┴───────────┘



In [115]:
with duckdb.connect('./src/db.duckdb') as con:
    con.sql("drop type quality_class")
    con.sql("create type quality_class as ENUM ('star', 'good', 'average', 'bad')")
    con.sql("drop type films")
    con.sql("""create type films as struct(
                film text,
                votes integer,
                rating real,
                filmid text)""")
    con.sql("DROP TABLE actors")
    con.sql("""CREATE TABLE actors (
            actor text,
            films films[],
            quality_class quality_class,
            is_active bool,
            year integer
            )""")

In [116]:
with duckdb.connect('./src/db.duckdb') as con:
    con.sql("""
            insert into actors
            with this_year_dupe as (
        select
            *,
            avg(rating) over (partition by actorid) avg_rating,
            array_agg(ROW(film, votes, rating, filmid)::films) over (partition by actorid) as  films,
            --struct_pack(ROW(film, votes, rating, filmid)::films) over (partition by actorid) as  films,
            row_number() over (partition by actorid) rn,
            case when actorid is not null then 1 else 0 end is_active
        from actor_films
        where
            year = 1970),
    last_year as (
            select
                *
        from actors
            where
                year = 1969
            ),
    this_year as (
            select
                *,
            case
                when ty.avg_rating > 8 then 'start'
            when ty.avg_rating > 7 then 'good'
            when ty.avg_rating > 6 then 'average'
            else 'bad'
        end::quality_class as quality_class,
        from this_year_dupe as ty
            where rn = 1
            )

    select
            coalesce(ty.actor, ly.actor) as actor,
            ty.films as films,
            ty.quality_class,
            ty.is_active,
            coalesce(ty.year, ly.year + 1) as year
    from this_year ty
            full outer join last_year ly
            on ty.actor = ly.actor

""").show()

AttributeError: 'NoneType' object has no attribute 'show'

In [40]:
with duckdb.connect('./src/db.duckdb') as con:
    con.sql("""
            select
                min(year)
        from actor_films
""").show()

┌─────────────┐
│ min("year") │
│    int32    │
├─────────────┤
│        1970 │
└─────────────┘



In [104]:
with duckdb.connect('./src/db.duckdb') as con:
    con.sql("""
            select
                *
            from actor_films
            where
                year = 1970
            """).show()

┌─────────────────────┬───────────┬─────────────────────────────┬───────┬───────┬────────┬───────────┐
│        actor        │  actorid  │            film             │ year  │ votes │ rating │  filmid   │
│       varchar       │  varchar  │           varchar           │ int32 │ int32 │ float  │  varchar  │
├─────────────────────┼───────────┼─────────────────────────────┼───────┼───────┼────────┼───────────┤
│ Brigitte Bardot     │ nm0000003 │ The Bear and the Doll       │  1970 │   431 │    6.4 │ tt0064779 │
│ Brigitte Bardot     │ nm0000003 │ Les novices                 │  1970 │   219 │    5.1 │ tt0066164 │
│ Ingrid Bergman      │ nm0000006 │ A Walk in the Spring Rain   │  1970 │   696 │    6.2 │ tt0066542 │
│ Bette Davis         │ nm0000012 │ Connecting Rooms            │  1970 │   585 │    6.9 │ tt0066943 │
│ Olivia de Havilland │ nm0000014 │ The Adventurers             │  1970 │   656 │    5.5 │ tt0065374 │
│ Kirk Douglas        │ nm0000018 │ There Was a Crooked Man...  │  1970 │

In [117]:
with duckdb.connect('./src/db.duckdb') as con:
    con.sql("""
            select
                *
            from actors
            where
                year = 1970
            """).show()

┌─────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────┬───────────┬───────┐
│        actor        │                                                                                                                                                                                                     films                                                                                                                                                                                                      │             quality_class              │ is_active │ year  │
│       varchar       │     